# Modelo de Intercambio Aleatorio de Activos (RAE)

## Sistema económico cerrado con conservación de dinero

El modelo utilizado es el de [Dragulescu & Yakovenko (2000)](https://doi.org/10.1007/s100510070114)

### Descripción del modelo

Los agentes económicos interaccionan por pares mediante transacciones monetarias. 

Solo se puede dar o recibir dinero en las transacciones.

Los agentes perdedores se etiquetan con el indice $i$ mientras que los ganadores con el indice $j$.

Sean $m$ y $m'$ la cantidad de dinero que posee un agente, antes y después de la transacción la cual se da según:

$$ m_i \rightarrow m'_i = m_i - \Delta m$$
$$ m_j \rightarrow m'_j = m_j + \Delta m$$

donde $\Delta m$ es la cantidad de dinero que el agente perdedor le transfirió al agente ganador después de la transacción.


Algunos supuestos:
- El dinero se conserva, global y localmente, $m_i + m_j = m'_i + m'_j $.
- El balance de los ganadores no puede ser negativo, $m_j  \ge 0 \ \forall j $.
- Una transacción se da solamente cuando el agente "perdedor" tiene suficiente dinero para pagar, $m_i  \ge \Delta m$.
- No hay crédito ni deuda.


La mecánica estadística predice que el sistema económico alcanzara la distribución estacionaria de Boltzmann-Gibbs,

$$ P(m) = \frac{1}{T} e^{-m/T} $$

donde 

$$ T = \langle m \rangle = \frac{M}{N} $$

es la _"Temperatura del dinero"_, y es igual a la cantidad promedio de dinero. $M$ es la cantidad total de dinero en el sistema, y $N$ es el numero total de agentes en el sistema. 


## Cómo se realizan las simulaciones

Las simulaciones se realizan en el siguiente orden:

- Inicialmente todos los agentes tienen la misma cantidad de dinero.
- Se elige un par $(i, j)$ de agentes al azar, y e realiza la transacción, es decir, se transfiere la cantidad $\Delta m$.
- Se repite el paso anterior varias veces.



En este trabajo se realizan las simulaciones con cada una de las siguientes reglas de intercambio:

- **Regla 1**: Intercambio de una pequeña cantidad constante de dinero, $\Delta m = 1$.
- **Regla 2**: Intercambio de una fracción aleatoria $0 \le \nu \le 1$ del dinero promedio del par, $\Delta m = \nu \left( m_i + m_j \right)/2$
- **Regla 3**: Intercambio de una fracción aleatoria $0 \le \nu \le 1$ del dinero promedio del sistema,  $\Delta m = \nu T$

Observese que estas reglas tienen simetría de inversión temporal, ya que son invariantes ante una permutación de los índices $i$ y $j$.

## Paquetes a utilizar 

In [1]:
using Random, Distributions, CairoMakie, LaTeXStrings

### Condiciones iniciales

Los parámetros para las simulaciones son: $N = 5 \times 10^{5}$ y $M = 5 \times 10^{6}$. Por lo tanto, al principio de las simulaciones, los agentes contarán con la misma cantidad de dinero $\left\langle m \right\rangle = \frac{M}{N} = 10$.

In [2]:
N_exp = 5
N = 5 * 10^N_exp     

M_exp = 6
M = 5 * 10^M_exp

T = M/N 

10.0

#### Reglas de intercambio 

In [3]:
exchange_rules = ["small_constant", "random_pair_average", "random_system_average"]

3-element Vector{String}:
 "small_constant"
 "random_pair_average"
 "random_system_average"

In [4]:
mutable struct Agent
  money::Float64
end

In [18]:
p = Agent(3)

Agent(3.0)

In [19]:
p.money

3.0